In [0]:
%run "../utils/mount_configuration"

In [0]:
%run "../utils/incremental_load"

In [0]:
from pyspark.sql.types import StructType, StructField, LongType, StringType, ArrayType, DoubleType

tickers_dividend_schema = StructType([
    StructField('request_id', StringType(), True), 
    StructField('results', ArrayType(StructType([
        StructField('cash_amount', DoubleType(), True), 
        StructField('currency', StringType(), True), 
        StructField('declaration_date', StringType(), True), 
        StructField('dividend_type', StringType(), True), 
        StructField('ex_dividend_date', StringType(), True), 
        StructField('frequency', LongType(), True), 
        StructField('id', StringType(), True), 
        StructField('pay_date', StringType(), True), 
        StructField('record_date', StringType(), True), 
        StructField('ticker', StringType(), True)]), True), True), 
    StructField('status', StringType(), True)])

In [0]:
tickers_dividend_df = spark.read \
    .schema(tickers_dividend_schema) \
    .json(f"{raw_folder_path}/dividend_data/*")

In [0]:
display(tickers_dividend_df)

In [0]:
from pyspark.sql.functions import explode, to_date

tickers_dividend_explode_df = tickers_dividend_df.select(explode("results").alias("result")) \
                                                .withColumn("pay_date", to_date("result.pay_date")) \
                                                .select("result.ticker", "result.cash_amount", "result.currency", 
                                                        "result.dividend_type", "pay_date")

display(tickers_dividend_explode_df)

In [0]:
mergeCondition = """target.ticker = source.ticker AND 
                    target.pay_date = source.pay_date"""

In [0]:
incrementalLoadDelta(input_df=tickers_dividend_explode_df, databaseName="engineering_processed", tableName="tickers_dividend", 
                     folderPath=processed_folder_path, partitionField="ticker",mergeCondition=mergeCondition)

In [0]:
%sql
SELECT * FROM engineering_processed.tickers_dividend;